In [11]:
import ipywidgets as widgets
from IPython.display import display, HTML

def calculate_change_euro(cost, amount_given):
    """
    Calculate the breakdown of change into Euro coins and bills.
    
    Args:
        cost (float): Cost of the item
        amount_given (float): Amount of money provided
    
    Returns:
        dict/str: Dictionary with results or error message
    """
    # Validation
    if cost < 0 or amount_given < 0:
        return "Error: Cost and amount given must be positive numbers."
    
    if amount_given < cost:
        return "Error: Amount given must be at least as much as the cost."
    
    # Calculate change in cents to avoid floating-point issues
    change = round(amount_given - cost, 2)
    total_cents = int(change * 100)
    
    # Euro denominations in cents (largest to smallest)
    denominations = [
        (50000, "500€ bill"),
        (20000, "200€ bill"),
        (10000, "100€ bill"),
        (5000, "50€ bill"),
        (2000, "20€ bill"),
        (1000, "10€ bill"),
        (500, "5€ bill"),
        (200, "2€ coin"),
        (100, "1€ coin"),
        (50, "50c coin"),
        (20, "20c coin"),
        (10, "10c coin"),
        (5, "5c coin"),
        (2, "2c coin"),
        (1, "1c coin")
    ]
    
    breakdown = {}
    remaining = total_cents
    
    # Greedy algorithm: use largest denominations first
    for value, name in denominations:
        count = remaining // value
        if count > 0:
            breakdown[name] = count
            remaining %= value
    
    return {
        'change': change,
        'breakdown': breakdown,
        'total_cents': total_cents
    }

# =================== GUI COMPONENTS ===================
# Style configuration
input_style = {'description_width': '120px'}
main_layout = widgets.Layout(width='450px', padding='5px')

# Input widgets
cost_input = widgets.FloatText(
    value=0.0,
    description='Cost (€):',
    style=input_style,
    layout=widgets.Layout(width='300px')
)

amount_input = widgets.FloatText(
    value=0.0,
    description='Amount Given (€):',
    style=input_style,
    layout=widgets.Layout(width='300px')
)

calculate_button = widgets.Button(
    description='Calculate Change',
    button_style='success',
    icon='euro-sign',
    layout=widgets.Layout(width='300px', margin='10px 0 0 120px')
)

# Output display area
output_area = widgets.Output(
    layout=widgets.Layout(
        width='400px',
        border='2px solid #0066cc',
        border_radius='5px',
        padding='15px',
        margin='15px 0'
    )
)

# =================== EVENT HANDLER ===================
def on_calculate_clicked(b):
    with output_area:
        output_area.clear_output()
        try:
            cost = cost_input.value
            amount_given = amount_input.value
            
            result = calculate_change_euro(cost, amount_given)
            
            if isinstance(result, dict):
                change_amount = result['change']
                breakdown = result['breakdown']
                
                # Display results with formatting
                display(HTML(f"""
                <div style='font-family: Arial, sans-serif;'>
                    <h3 style='color:#0066cc; margin-top:0;'>💶 Change Calculation Result</h3>
                    <div style='background-color:#f0f8ff; padding:10px; border-radius:5px; margin-bottom:10px;'>
                        <strong>Total Change Due: €{change_amount:.2f}</strong>
                    </div>
                    <div style='font-size:0.9em;'>
                """))
                
                if breakdown:
                    print("=" * 45)
                    print("Optimal Denomination Breakdown:")
                    print("=" * 45)
                    
                    for denom, count in breakdown.items():
                        print(f"{denom:>15}: {count}")
                else:
                    print("✓ No change needed - exact payment!")
                
                print("=" * 45)
                print(f"Verification: €{result['total_cents']/100:.2f} ✓")
                display(HTML("</div>"))
                
            else:
                # Display error message
                display(HTML(f"<div style='color:red; font-weight:bold;'>{result}</div>"))
                
        except Exception as e:
            display(HTML(f"<div style='color:red;'>An error occurred: {e}</div>"))

# Link button to function
calculate_button.on_click(on_calculate_clicked)

# =================== DISPLAY GUI ===================
display(HTML("<h2 style='color:#0066cc;'>🇪🇺 Euro Change Return Calculator</h2>"))
display(widgets.VBox([
    cost_input,
    amount_input,
    calculate_button,
    output_area
], layout=main_layout))